<a href="https://colab.research.google.com/github/thursdaydreaming/whitescan/blob/main/categorizing_v1_(Okt%2C_TF_IDF_%EC%9D%B4%EC%9A%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
%matplotlib inline

!pip install scikit-learn==0.21.1
!pip install konlpy

In [159]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import csv
import warnings
warnings.filterwarnings("ignore")
from konlpy.tag import Okt
import numpy as np

In [160]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 테스트하려면 주신 데이터셋(xlrx)를 csv로 변환해서 새로 저장해야 함! Sheet1만 추출!

In [161]:
data = pd.read_csv("drive/MyDrive/Sheet1.csv", sep=',')
print(data)

       카테고리          키워드
0     의류/잡화          원피스
1     의류/잡화  써스데이아일랜드원피스
2     의류/잡화        여름원피스
3     의류/잡화         블라우스
4     의류/잡화        남자반팔티
...     ...          ...
2995    화장품         프라이머
2996    화장품    vdl아이프라이머
2997    화장품       아이프라이머
2998    화장품   맥라이트풀c프라이머
2999    화장품       모공프라이머

[3000 rows x 2 columns]


In [162]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['키워드'])
doc_similarities = (tfidf_matrix * tfidf_matrix.T)

print(doc_similarities.toarray())
#여기서 1은 자기 자신과의 유사도, 1은 완전 일치 0은 완전 불일치

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [163]:
#명사 추출 후 유사도 재검사 (개선)

okt = Okt()

doc_nouns_list = [' '.join(okt.nouns(doc)) for doc in data['키워드']]
print(doc_nouns_list)

tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)
doc_nouns_similarities = (tfidf_matrix * tfidf_matrix.T)

print(doc_nouns_similarities.toarray())

['원피스', '써스데이 아일랜드 원피스', '여름 원피스', '블라우스', '남자 티', '롱 원피스', '여성 점프수트', '라코스테 원피스', '메종 키츠네 팔', '쥬 쥬디', '플라스틱 아일랜드 원피스', '컷 원피스', '남자 반바지', '뷔 스티 원피스', '폴로 티', '린넨 원피스', '앤 원피스', '여름 가디건', '나이키 반바지', '케네 스레이 디 원피스', '시슬리 원피스', '스포츠 브라', '브라 렛', '여성 원피스', '바스락 원피스', '나이키 팔', '고트 원피스', '점프수트', '반바지', '나이키 티', '쉬폰 원피스', '미샤 원피스', '잠옷', '모조 스핀 원피스', '로엠 원피스', '쉬즈 미스 원피스', '여성 블라우스', '아디다스 반바지', '랩 원피스', '티셔츠', '라코스테 티', '아미 티', '보브 원피스', '샤틴 원피스', '여성 반바지', '나이키 바람막이', '타미힐피거 팔', '에고 이스트 원피스', '스톤아일랜드 팔', '여름 롱 원피스', '써스데이 아일랜드', '여름 잠옷', '스 투시 팔', '티', '여성 린넨 자켓', '여성 티셔츠', '스퀘어 넥 원피스', '쥬 쥬디 원피스', '남자 버핏 팔', '셔츠 원피스', '데님 원피스', '티셔츠', '베네통 원피스', '여름 블라우스', '듀 원피스', '남자 린넨 셔츠', '모조 스핀', '바람막이', '써스데이 아일랜드 블라우스', '스커트', '리스트 원피스', '고트 원피스', '냉장고바지', '리츠 원피스', '라코스테 카라 티', '하객 원피스', '미샤', '여성 여름 자켓', '펀칭 원피스', '에고 이스트', '자켓', '노 와이어 브라', '남자 팬티', '플라스틱 아일랜드', '메종 키츠네', '컷 블라우스', '남성 반바지', '퍼프 블라우스', '헤지 스 남성 티셔츠', '폴로 카라 티', '남자 셔츠', '우비', '더 머 반바지', '커플 잠옷', '캘빈 클라인 팔', '올리비아 로렌', '로엠', '여름

#**위의 것은 테스트용, 지금부터가 진짜**


In [164]:
# 3개의 카테고리, 개별 카테고리마다 1000개의 샘플데이터
# 상호 간 유사도 측정 뒤, 총 3000개의 샘플 데이터 중 유사도 상위 100개의 일치 비율 파악

# 데이터 프레임에 행추가해서 ID 값 부여
ID=[]

for i in range(1,len(data)+1):
  ID.append(i)
  i+=1
#print(ID)

data['ID']=ID
#data.head(10)

#ID 인덱스화
#data=data.set_index('ID')
data.head(10)

,카테고리,키워드,ID
0,의류/잡화,원피스,1
1,의류/잡화,써스데이아일랜드원피스,2
2,의류/잡화,여름원피스,3
3,의류/잡화,블라우스,4
4,의류/잡화,남자반팔티,5
5,의류/잡화,롱원피스,6
6,의류/잡화,여성점프수트,7
7,의류/잡화,라코스테원피스,8
8,의류/잡화,메종키츠네반팔,9
9,의류/잡화,쥬시쥬디,10


In [165]:
# 명사 추출한 데이터만 df에 남기고 모두 삭제

#data['keyword']=doc_nouns_list
#del data['키워드']

# 유사도도 df에 넣었다가 일단 뺌
#data['similarity']=doc_nouns_similarities
#del data['similarity']

data.head(10)

,카테고리,키워드,ID
0,의류/잡화,원피스,1
1,의류/잡화,써스데이아일랜드원피스,2
2,의류/잡화,여름원피스,3
3,의류/잡화,블라우스,4
4,의류/잡화,남자반팔티,5
5,의류/잡화,롱원피스,6
6,의류/잡화,여성점프수트,7
7,의류/잡화,라코스테원피스,8
8,의류/잡화,메종키츠네반팔,9
9,의류/잡화,쥬시쥬디,10


In [166]:
# 카테고리 값 코드화(의류/잡화:0, 음식:1, 화장품:2)
#data.loc[(data.카테고리 == '의류/잡화'), '카테고리']=0
#data.loc[(data.카테고리 == '음식'), '카테고리']=1
#data.loc[(data.카테고리 == '화장품'), '카테고리']=2
#data.head(10)

#생각해보니 코드화하면 카테고리와의 유사도분류가 힘든가? 이 부분은 생각해봐야 할 듯

In [167]:
#위에 만들어둔거 활용해서 id와 similarity 뽑아 상위 10개 리스트에 정리하는 함수 다시 짜야함

import numpy as np 
print(type(data))

#이렇게 하면 dataframe (명사화시킨 열 추가 안한다면)

<class 'pandas.core.frame.DataFrame'>


In [168]:
#명사 추출 후 유사도 재검사 (개선)

okt = Okt()

doc_nouns_list = [' '.join(okt.nouns(doc)) for doc in data['키워드']]
#print(doc_nouns_list)

tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)
doc_similarities = (tfidf_matrix * tfidf_matrix.T) #여기까지 밑에 그대로 씀

print(doc_similarities.toarray())

[[1.         0.38630541 0.61523159 ... 0.         0.         0.        ]
 [0.38630541 1.         0.23766729 ... 0.         0.         0.        ]
 [0.61523159 0.23766729 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.57759718 0.83653504]
 [0.         0.         0.         ... 0.57759718 1.         0.69046382]
 [0.         0.         0.         ... 0.83653504 0.69046382 1.        ]]


In [169]:
print(doc_nouns_similarities[0])
print(type(doc_nouns_similarities))

  (0, 487)	0.5194354087394726
  (0, 483)	0.47000270692803403
  (0, 474)	0.3539124465253216
  (0, 473)	0.4947126385677089
  (0, 465)	0.5934643783924363
  (0, 460)	0.5122074757959613
  (0, 459)	0.3685822052688403
  (0, 439)	0.4518546225452886
  (0, 437)	1.0
  (0, 431)	0.5194354087394726
  (0, 418)	0.35801543541720293
  (0, 415)	1.0
  (0, 411)	0.4518546225452886
  (0, 407)	0.4518546225452886
  (0, 400)	0.5515961809669935
  (0, 399)	0.47000270692803403
  (0, 390)	1.0
  (0, 388)	0.47000270692803403
  (0, 384)	0.4518546225452886
  (0, 383)	0.5704877753597173
  (0, 379)	0.5557788511204279
  (0, 375)	0.5318903607821118
  (0, 370)	0.5424270933371224
  (0, 369)	0.4518546225452886
  (0, 365)	0.40262328147060517
  :	:
  (0, 49)	0.6152315877893184
  (0, 47)	0.36393981042052553
  (0, 43)	0.47000270692803403
  (0, 42)	0.4947126385677089
  (0, 38)	1.0
  (0, 35)	0.36393981042052553
  (0, 34)	0.48365382326465584
  (0, 33)	0.36393981042052553
  (0, 31)	0.5704877753597173
  (0, 30)	0.47000270692803403
  (